# Generating embeddings for SIB SPARQL queries

Using `fastembed` library, FlagEmbedding large 1.5 embedding model, and Qdrant vectorstore

Example code for loading the large Babel concepts synonyms dataset to Qdrant: https://github.com/vemonet/concept-resolver/blob/main/src/babel_load.py

Questions:
* Which info can I get from the UniProt endpoint?
* Give me an example to access cross references from the UniProt SPARQL endpoint to all the databases available in the endpoint

In [1]:
from expasy_api.vectordb import init_vectordb

# system_prompt = """You are Expasy, an assistant that helps users to query the databases from the Swiss Institute of Bioinformatics, such as UniProt or Bgee.
# When writing the SPARQL query try to factorize the predicates/objects of a subject as much as possible, so that the user can understand the query and the results.
# """
# examples_prompt: str = "Here are a list of questions and queries that Expasy has learned to answer, use them as base when answering the question from the user:"

init_vectordb("localhost")

/home/vemonet/dev/expasy/expasy-api/.venv/expasy-api/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
special_tokens_map.json: 100%|██████████| 695/695 [00:00<00:00, 1.57MB/s]

config.json: 100%|██████████| 742/742 [00:00<00:00, 2.51MB/s]

tokenizer_config.json: 100%|██████████| 1.24k/1.24k [00:00<00:00, 750kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 2.42MB/s]
model.onnx: 100%|██████████| 1.34G/1.34G [06:38<00:00, 3.36MB/s]
Fetching 5 files: 100%|██████████| 5/5 [06:38<00:00, 79.77s/it] 


Found 60 queries for https://sparql.uniprot.org/sparql/


/home/vemonet/dev/expasy/expasy-api/src/expasy_api/vectordb.py:55: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, "html.parser")


Found 19 queries for https://www.bgee.org/sparql/
Found 13 queries for https://sparql.omabrowser.org/sparql/
Done generating embeddings for 92 queries
